<a href="https://colab.research.google.com/github/AllanKamimura/AI/blob/master/buzzleixo/buzzleixo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Inicializador

In [ ]:
#importa as bibliotecas ai
import os
import zipfile
import datetime
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from google.colab import files
from tf.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#baixa os arquivos do dataset
!wget --no-check-certificate \
    https://github.com/AllanKamimura/AI/blob/master/buzzleixo/train.zip?raw=true \
    -O /tmp/Train.zip

### Estrutura dos dados
```
── Train
    ├── Buzz
    │   ├── Buzz1.jpg
    │   ├── Buzz2.jpg
    │   └──  ...
    └── Juju
        ├── Juju1.jpg
        ├── Juju2.jpg
        └──  ...
```

In [ ]:
local_zip = '/tmp/Train.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

# Directory with our training buzz pictures
train_buzz_dir = os.path.join('/tmp/Train/Buzz')

# Directory with our training juju pictures
train_juju_dir = os.path.join('/tmp/Train/Juju')

train_buzz_names = os.listdir(train_buzz_dir)
print(train_buzz_names[:10])

train_juju_names = os.listdir(train_juju_dir)
print(train_juju_names[:10])

print('total training buzz images:', len(os.listdir(train_buzz_dir)))
print('total training juju images:', len(os.listdir(train_juju_dir)))

In [ ]:
# All images will be rescaled by 1./255 + data augmentation

train_datagen = ImageDataGenerator(
    rescale = 1/255,
    rotation_range = 45,
    width_shift_range = 0.4,
    height_shift_range = 0.4,
    shear_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    zoom_range = 0.5,
    fill_mode = 'nearest')


# Flow training images in batches  using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/tmp/train',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size= 10,
        shuffle = True,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode = 'binary')

### Rede

In [ ]:
nodes = 32 

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D( nodes , (3,3), activation = "relu", input_shape = (300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D( 2*nodes, (3,3), activation = "relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D( 4*nodes, (3,3), activation = "relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D( 8*nodes, (3,3), activation = "relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense( 10*nodes, activation = "relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid")])
model.summary()


In [ ]:
model.compile(
    loss = "binary_crossentropy", 
    optimizer = "RMSprop", 
    metrics = ["accuracy"])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = 10,
    batch_size = 10,
    epochs = 25,
    verbose = 1)

### Faça o upload da imagem e veja o resultado

In [ ]:
uploaded = files.upload() #botão de fazer upload

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn #salva as imagens uploudadas aqui
  img = image.load_img(path, target_size=(300, 300)) #usa o keras.preprocess para abrir a imagem
  x = image.img_to_array(img) # transforma a imagem em um array
  x = np.expand_dims(x, axis=0) #aumenta as dimensões do array (batch_size, altura, largura, canal)

  images = np.vstack([x])
  classes = model.predict(images, batch_size = 10)
  print(classes[0])
  if classes[0] > 0.5:
    print(fn + " É UM ABACATE")
    %pylab inline
    img = image.load_img('/tmp/aleixo.jpg')
    plt.imshow(img)
    plt.show()
  else:
    print(fn + " É UM BUZZLATIR")
    %pylab inline
    img = image.load_img('/tmp/buzz.jpg')
    plt.imshow(img)
    plt.show()

### Salve seus pesos
obs os arquivos temporarios (/tmp/) serão deletados quando a sessão for encerrada

In [ ]:
#cria um savepoint com a hora num fuso horario estranho ai
def save():
    now = datetime.datetime.now() 
    now = "{}_{}_{}_{}_{}".format(now.year,now.month,now.day,now.hour,now.minute)
    model.save("/tmp/" + now + ".h5")
save()